RRF (Reciprocal Rank Fusion) 알고리즘 구현
    - 검색 결과를 '순위' 기반 통합 후 검색 정확도 개선
    - 중복 등장하는 문서에 높은 가중치 부여, 관련성 높은 문서를 최상단으로 이동

RRF는 여러 검색 결과 리스트(R)에 포함된 각 문서(d)에 대해 다음과 같이 점수를 계산함:

$$Score(d) = \sum_{r \in R} \frac{1}{k + r(d)}$$

- $r(d)$: 특정 검색 결과 리스트에서의 문서 d의 순위 (1-based index; 즉, 1부터 시작) 
- $k$ (ranking constant): 하위 순위 문서의 영향력을 조절하는 상수 (아래 예시에서는 60 사용)

In [1]:
from tqdm import tqdm
from score import get_hitn

In [2]:
"""
reciprocal rank fusion 도입

https://www.alibabacloud.com/blog/enhancing-search-accuracy-with-rrfreciprocal-rank-fusion-in-alibaba-cloud-elasticsearch-8-x_601024
https://learn.microsoft.com/ko-kr/azure/search/hybrid-search-ranking

1. 테스트 결과 (정답, 예측)을 입력으로 받아
2. reciprocal rank fusion (RRF) 적용한 예측결과를 반환하는 함수 작성
3. 함수의 반환값은 예측 포맷과 동일해야 함 
    ex) ['[0]', '[0]', '[0]', '[1477]', '[2267]', '[2495]', '[0]', '[2536]']
4. RRF 적용 후 테스트 결과 생성

"""

"\nreciprocal rank fusion 도입\n\nhttps://www.alibabacloud.com/blog/enhancing-search-accuracy-with-rrfreciprocal-rank-fusion-in-alibaba-cloud-elasticsearch-8-x_601024\nhttps://learn.microsoft.com/ko-kr/azure/search/hybrid-search-ranking\n\n1. 테스트 결과 (정답, 예측)을 입력으로 받아\n2. reciprocal rank fusion (RRF) 적용한 예측결과를 반환하는 함수 작성\n3. 함수의 반환값은 예측 포맷과 동일해야 함 \n    ex) ['[0]', '[0]', '[0]', '[1477]', '[2267]', '[2495]', '[0]', '[2536]']\n4. RRF 적용 후 테스트 결과 생성\n\n"

In [3]:
with open("logs.txt", "r") as f:
    data = [line.strip() for line in f.readlines() if line.strip()]

data[:10]

["0//['[0]', '[0]', '[37, 38]', '[0]', '[3264]', '[1477]', '[0, 1]', '[2267]', '[2495]', '[0]', '[1329]', '[3274]', '[2536]']",
 "0//['[0]', '[2121]', '[1477]', '[757]', '[2416, 2417]', '[0]', '[0]', '[0]', '[3128]', '[0, 1]', '[3595]', '[4176]', '[3595]']",
 "1//['[1]', '[1]', '[1]', '[1]', '[1]', '[1, 2]', '[3969]', '[1]', '[1]', '[1]', '[1]']",
 "1//['[1]', '[1]', '[1]', '[1659, 1660]', '[1660]', '[1660]', '[1660]', '[1660]', '[0, 1]', '[2179]', '[2179]', '[1660]']",
 "1//['[3447]', '[4481]', '[3895]', '[758]', '[1]', '[2813]', '[1375]', '[4602]', '[1019]', '[2244]', '[3715]', '[1]', '[4602, 4603]', '[3201]']",
 "2//['[2]', '[2]', '[2]', '[2790]', '[2120]', '[4284]', '[1, 2]', '[4717]', '[2649]', '[1057]', '[18]', '[4059]', '[380]']",
 "2//['[903]', '[2]', '[3987]', '[1475]', '[1, 2]', '[2429]', '[3987]', '[727, 728]', '[2368, 2369]', '[2276]', '[1833, 1834]', '[261]', '[3843]', '[3843]']",
 "3//['[3]', '[3]', '[3]', '[2094]', '[3]', '[493]', '[2094]', '[1230]', '[1230]', '[3769, 37

In [4]:
hit1_score_sum = 0

for each in tqdm(data):
    answer, candidate = each.split("//")
    
    hit1_score = get_hitn(candidates=candidate, answer=answer, N=1)
    
    hit1_score_sum += hit1_score

hit1_score_avg = hit1_score_sum / len(data)
print("hit1 score:", hit1_score_avg)

100%|██████████| 10160/10160 [00:00<00:00, 44666.79it/s]

hit1 score: 0.7139763779527559


In [5]:
import ast
from collections import defaultdict

def RRF_rerank(data, k = 60): 
    # 1. 데이터 파싱
    answer_str, candidates_str = data.split('//')
    # answer_str은 현재 랭킹 로직에서 사용되지 않으나, 추후 성능 평가를 위해 분리함
    
    # 리스트 문자열 파싱
    raw_candidates = ast.literal_eval(candidates_str)
    
    # 2. 전처리
    candidates = []
    for item in raw_candidates:
        if ',' in item:
            # '[3, 4]' -> [3, 4] 변환
            parsed_items = ast.literal_eval(item)
            candidates.extend(parsed_items)
        else:
            # '[3]' -> 3 변환
            candidates.append(int(item.strip('[]')))

    # 3. RRF 점수 계산
    doc_scores = defaultdict(float)
    
    # 입력 리스트를 순회하며 점수 누적; 입력된 리스트가 이미 여러 검색기의 결과가 합쳐진 순서라고 가정
    for rank, doc_id in enumerate(candidates):
        # RRF Formula: 1 / (k + rank) (rank starts from 1)
        score = 1.0 / (k + (rank + 1))
        doc_scores[doc_id] += score

    # 4. 점수 기준 내림차순 정렬
    # items()는 (key, value) 튜플을 반환 -> value(x[1]) 기준으로 정렬
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    
    # 5. 결과 포맷팅 (요구사항에 맞게 변환)
    final_ranking = [f"[{doc_id}]" for doc_id, score in sorted_docs]
    final_scores = [score for doc_id, score in sorted_docs] # 점수 확인용

    return (final_ranking, final_scores) # 요구사항: 예측 포맷 리스트만 반환

# --- Test ---
example_data = "3//['[3]', '[3]', '[3593]', '[3]', '[2206]', '[3]', '[3]', '[3, 4]', '[2206]', '[4050]', '[3997]', '[193]', '[2610]', '[422]', '[3593]']"

result = RRF_rerank(example_data, k=60)
print(result)

(['[3]', '[2206]', '[3593]', '[4]', '[4050]', '[3997]', '[193]', '[2610]', '[422]'], [0.09293024551980003, 0.02967032967032967, 0.029030910609857977, 0.014492753623188406, 0.014084507042253521, 0.013888888888888888, 0.0136986301369863, 0.013513513513513514, 0.013333333333333334])


Last Updated: January 21, 2026 \
Original Version: June 9, 2025